In [1]:
from data_utils import MergedMelHarmDataset, MLMCollator
import os
import numpy as np
from harmony_tokenizers_m21 import ChordSymbolTokenizer, RootTypeTokenizer, \
    PitchClassTokenizer, RootPCTokenizer, GCTRootPCTokenizer, \
    GCTSymbolTokenizer, GCTRootTypeTokenizer, MelodyPitchTokenizer, \
    MergedMelHarmTokenizer
from torch.utils.data import DataLoader
from transformers import RobertaConfig, RobertaForMaskedLM
import torch
from torch.optim import AdamW
from tqdm import tqdm

/home/maximos/anaconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root_dir = '/media/datadisk/datasets/hooktheory_xmls_transposed'
# root_dir = 'data/gjt_melodies/Library_melodies/'
# root_dir = '/media/maindisk/maximos/data/gjt_melodies/Library_melodies/'
# data_files = []

# # Walk through all subdirectories and files
# for dirpath, _, filenames in os.walk(root_dir):
#     for file in filenames:
#         if file.endswith('.xml') or file.endswith('.mxl'):
#             full_path = os.path.join(dirpath, file)
#             data_files.append(full_path)

# print('Total files from Hook Theory dataset:', len(data_files))

In [3]:
chordSymbolTokenizer = ChordSymbolTokenizer.from_pretrained('saved_tokenizers/ChordSymbolTokenizer')
rootTypeTokenizer = RootTypeTokenizer.from_pretrained('saved_tokenizers/RootTypeTokenizer')
pitchClassTokenizer = PitchClassTokenizer.from_pretrained('saved_tokenizers/PitchClassTokenizer')
rootPCTokenizer = RootPCTokenizer.from_pretrained('saved_tokenizers/RootPCTokenizer')
gctRootPCTokenizer = GCTRootPCTokenizer.from_pretrained('saved_tokenizers/GCTRootPCTokenizer')
gctSymbolTokenizer = GCTSymbolTokenizer.from_pretrained('saved_tokenizers/GCTSymbolTokenizer')
gctRootTypeTokenizer = GCTRootTypeTokenizer.from_pretrained('saved_tokenizers/GCTRootTypeTokenizer')
melodyPitchTokenizer = MelodyPitchTokenizer.from_pretrained('saved_tokenizers/MelodyPitchTokenizer')

In [4]:
m_chordSymbolTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, chordSymbolTokenizer)
m_rootTypeTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, rootTypeTokenizer)
m_pitchClassTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, pitchClassTokenizer)
m_rootPCTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, rootPCTokenizer)
m_gctRootPCTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, gctRootPCTokenizer)
m_gctSymbolTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, gctSymbolTokenizer)
m_gctRootTypeTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, gctRootTypeTokenizer)

In [5]:
tokenizer = m_chordSymbolTokenizer

dataset = MergedMelHarmDataset(root_dir, tokenizer, max_length=512, pad_to_max_length=True, num_bars=8)
collator = MLMCollator(tokenizer)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collator)


In [6]:
# m_len = 0
# len_all = []
# with tqdm(dataloader, unit='batch') as tepoch:
#     for b in tepoch:
#         len_all.append( b['input_ids'].shape[1] )
#         if b['input_ids'].shape[1] > m_len:
#             m_len = b['input_ids'].shape[1]
#         tepoch.set_postfix(loss=m_len, accuracy=np.mean(len_all))

In [7]:
print(tokenizer.vocab[ tokenizer.harmony_tokenizer.start_harmony_token ])
print(tokenizer.vocab['<bar>'])
b = next(iter(dataloader))
print(b)

196
6


/home/maximos/anaconda3/envs/torch/lib/python3.11/site-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
/home/maximos/anaconda3/envs/torch/lib/python3.11/site-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
In /home/maximos/anaconda3/envs/torch/lib/python3.11/site-packages/matplotlib/mpl-data/stylelib/seaborn-v0_8-notebook.mplstyle: .flat is deprecated.  Call .flatten() instead
In /home/maximos/anaconda3/envs/torch/lib/python3.11/site-packages/matplotlib/mpl-data/stylelib/seaborn-v0_8-talk.mplstyle: .flat is deprecated.  Call .flatten() instead
/home/maximos/anaconda3/envs/torch/lib/python3.11/site-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(clas

{'input_ids': tensor([[  2,   6, 183,  ...,   1,   1,   1],
        [  5,   6, 180,  ...,   1,   1,   1],
        [  2,   6, 180,  ...,   1,   1,   1],
        ...,
        [  5,   6, 180,  ...,   1,   1,   1],
        [  2,   6, 180,  ...,   1,   1,   1],
        [  2,   5, 180,  ...,   1,   1,   1]]), 'labels': tensor([[-100, -100, -100,  ..., -100, -100, -100],
        [   2, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        ...,
        [   2, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100,    6, -100,  ..., -100, -100, -100]])}


In [8]:
print(b['input_ids'].shape)
print(b['labels'].shape)

torch.Size([32, 512])
torch.Size([32, 512])


In [9]:
model_config = RobertaConfig(
    vocab_size=len(tokenizer.vocab),
    hidden_size=512,
    num_hidden_layers=8,
    num_attention_heads=8,
    pad_token_id=tokenizer.vocab[tokenizer.pad_token],
    bos_token_id=tokenizer.vocab[tokenizer.bos_token],
    eos_token_id=tokenizer.vocab[tokenizer.eos_token],
    mask_token_id=tokenizer.vocab[tokenizer.mask_token],
    max_position_embeddings=512,
)

In [10]:
model = RobertaForMaskedLM(model_config)
model.train()

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(545, 512, padding_idx=1)
      (position_embeddings): Embedding(512, 512, padding_idx=1)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-7): 8 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): 

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
for epoch in range(3):  # Number of epochs
    train_loss = 0
    running_loss = 0
    batch_num = 0
    running_accuracy = 0
    train_accuracy = 0
    with tqdm(dataloader, unit='batch') as tepoch:
        tepoch.set_description(f"Epoch {epoch} | trn")
        for batch in tepoch:
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)
            
            outputs = model(input_ids, labels=labels)
            loss = outputs.loss
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update loss
            batch_num += 1
            running_loss += loss.item()
            train_loss = running_loss/batch_num
            # accuracy
            predictions = outputs.logits.argmax(dim=-1)
            mask = labels != -100
            running_accuracy += (predictions[mask] == labels[mask]).sum().item()/mask.sum().item()
            train_accuracy = running_accuracy/batch_num
            
            tepoch.set_postfix(loss=train_loss, accuracy=train_accuracy) # tepoch.set_postfix(loss=loss.item(), accuracy=100. * accuracy)

Epoch 0 | trn:   0%|          | 0/475 [00:00<?, ?batch/s]/home/maximos/anaconda3/envs/torch/lib/python3.11/site-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
Epoch 0 | trn:  24%|██▍       | 114/475 [08:21<26:27,  4.40s/batch, accuracy=0.164, loss=4.02]


KeyboardInterrupt: 